In [1]:
%pip install hypothesis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.2/387.2 KB 11.6 MB/s eta 0:00:00
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [51]:
from hypothesis.strategies import from_regex
import random
from typing import Dict, Literal, Sequence, Tuple

N = 100

youtube_url = from_regex(r"^((https://www\.youtube\.com/watch\?v=)|(https://youtu\.be/))[A-Za-z0-9_]+$")
lang = random.choice(["Russian", "English", "Spanish", "Ukrainian", "German", "Portuguese"])
method = random.choice(["Machine A", "Machine B", "Subtitles"])

Entity = Literal["language", "url", "method"]


def entities(url: str, lang: str, method: str) -> Dict[Entity, str]:
    return {
        "url": url,
        "method": method,
        "language": lang,
    }


def example(text: str, lookup: Dict[Entity, str]) -> Sequence[Tuple[str, Tuple[str, int, int]]]:
    print(f"text={text} lookup={lookup}")
    return [(entity, text.index(value), len(value)) for entity, value in lookup.items()]


# load(url, source_lang, method) -> (id, url)
def examples(n: int) -> Sequence[Tuple[str, Tuple[str, int, int]]]:
    urls = [youtube_url.example() for _ in range(n)]
    langs = [random.choice(["Russian", "English", "Spanish", "Ukrainian", "German", "Portuguese"]) for _ in range(n)]
    methods = [random.choice(["Machine A", "Machine B", "Subtitles"]) for _ in range(n)]

    for url, lang, method in zip(urls, langs, methods):
        yield random.choice([
            example(f"{url} with {lang} {method}", entities(url, lang, method)),
            example(f"{url} with {lang} transcript using {method}", entities(url, lang, method)),
            example(f"load {url} with {lang} {method}", entities(url, lang, method)),
            example(f"load {url} with {lang} transcript using {method}", entities(url, lang, method)),
            example(f"create transcript from {url} using {lang} {method}", entities(url, lang, method)),
            example(f"create transcript from {url} and use {lang} {method}", entities(url, lang, method)),
            example(f"Create {lang} transcript for {url} using {method}", entities(url, lang, method)),
            example(f"Create {lang} transcript for {url} from {method}", entities(url, lang, method)),
            example(f"transcribe {url} from {lang} using {method}", entities(url, lang, method)),
        ])


def load(n: int, split: float):
    for text, entities in examples(n):
        yield {
            "text": text,
            "language": "en-us",
            "dataset": "Train" if random.random() < split else "Test",
            "intent": "transcribe",
            "entities": [{"entityName": name, "offset": offset, "length": length} for name, offset, length in entities]
        }

list(examples(10))
# translate(transcript_id, target_lang) -> (id, url)
# Human: translate https://notion.so to English
# Assistant: Here you are: https://notion.so

# reset(transcript_id) -> (id, url)
# Human: reset transcript https://notion.so
# Human: reset https://notion.so


# dub(trasncript_id) -> (audio, video, subtitles)
# Human: dub https://notion.so 
# Human: dub using transcript https://notion.so 
# Human: dub using https://notion.so 

NameError: free variable 'youtube_url' referenced before assignment in enclosing scope

In [37]:
[generator.example().strip() for _ in range(100)]

['https://youtu.be/S',
 'https://www.youtube.com/watch?v=GGW',
 'https://youtu.be/r111WWrWT2Wm',
 'https://www.youtube.com/watch?v=Yy52bw',
 'https://youtu.be/cSq_cvsf',
 'https://youtu.be/WWb',
 'https://www.youtube.com/watch?v=og',
 'https://youtu.be/T111WWrWT2Wm',
 'https://youtu.be/O',
 'https://www.youtube.com/watch?v=ii11W7evPS',
 'https://youtu.be/2',
 'https://www.youtube.com/watch?v=8DVu',
 'https://www.youtube.com/watch?v=2MVpMbJN',
 'https://www.youtube.com/watch?v=H',
 'https://youtu.be/S0',
 'https://www.youtube.com/watch?v=r7eEoEq',
 'https://youtu.be/bbCwX',
 'https://youtu.be/0166stYAYrex9htS2N',
 'https://www.youtube.com/watch?v=m3HXnt1ES9',
 'https://youtu.be/11Pj',
 'https://youtu.be/6fVT',
 'https://www.youtube.com/watch?v=i',
 'https://youtu.be/5Sq_cvsf',
 'https://youtu.be/Y',
 'https://youtu.be/A1A7',
 'https://youtu.be/OPR',
 'https://youtu.be/27ZApl1M2lDUGN_',
 'https://youtu.be/z',
 'https://youtu.be/Yy57HkTGKpoWQez7',
 'https://www.youtube.com/watch?v=Q4bQ',
